In [27]:
import os
import json
from typing import Dict, List, Optional, Union, cast
from IPython.display import display
from ipywidgets import IntProgress
import requests
import pandas as pd
from bs4 import BeautifulSoup

import time
from requests import get
from env import github_token, github_username

## Get repo name

In [ ]:
repo_list = []

for i in range(1,10):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list.append(repo.text)
repo_list

In [ ]:
# after 1 minite
repo_list1 = []

for i in range(10,20):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list1.append(repo.text)
repo_list1

In [ ]:
repo_list2 = []

for i in range(20,30):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list2.append(repo.text)
repo_list2

In [ ]:
repo_list3 = []

for i in range(30,40):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list3.append(repo.text)
repo_list3

In [ ]:
repo_list4 = []

for i in range(40,50):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list4.append(repo.text)
repo_list4

In [ ]:
repo_list5 = []

for i in range(50,60):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list5.append(repo.text)
repo_list5

In [ ]:
repo_list6 = []

for i in range(60,70):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list6.append(repo.text)
repo_list6

In [ ]:
repo_list7 = []

for i in range(70,80):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list7.append(repo.text)
repo_list7

In [ ]:
repo_list8 = []

for i in range(80,90):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list8.append(repo.text)
repo_list8

In [ ]:
repo_list9 = []

for i in range(90,100):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list9.append(repo.text)
repo_list9

In [ ]:
repo_list10 = []

for i in range(100,110):
    response = get('https://github.com/search?p={}&q=inflation&type=Repositories'.format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for repo in soup.find_all('a', class_ = 'v-align-middle'):
        repo_list10.append(repo.text)
repo_list10

In [ ]:
final = repo_list + repo_list1 + repo_list2 + repo_list3 + repo_list4 + repo_list5 + repo_list6 + repo_list7 + repo_list8 + repo_list9 + repo_list10
len(final)

In [ ]:
df_final = pd.DataFrame(final)
df_final.rename(columns = {0 : 'repo_name'}, inplace = True)
df_final.head()

In [ ]:
df_final = df_final.drop_duplicates()
len(df_final)

In [ ]:
df_final.to_csv('repo_name.csv', index = False)

## get readme content

In [ ]:
df = pd.read_csv('repo_name.csv')
repo_list = df['repo_name'].to_list()

In [2]:
df.shape

(848, 1)

In [6]:
# query = 'inflation&type=Repositories'

In [13]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

REPOS = repo_list

def github_api_request(url: str) -> Union[List, Dict]:
    time.sleep(0.25)
    print(url)
    response = requests.get(url, headers=headers, timeout=60)
    response_data = response.json()
    if int(response.headers['X-RateLimit-Remaining']) < 10:
        print('Rate-limit exceeded. Slowing down.')
        time.sleep(60)
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data

def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )
    
def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )
    
def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""

def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    try:
        readme_contents = requests.get(get_readme_download_url(contents)).text
    except:
        readme_contents = ''
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }

def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    f = IntProgress(min=0, max=len(REPOS), description='Downloading Data')
    display(f)
    output = []
    for i, repo in enumerate(REPOS):
        f.value = i
        output.append(process_repo(repo))
    return output

def get_data(refresh=False):
    filename = './data.csv'
    if refresh or not os.path.isfile(filename):
        data = scrape_github_data()
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
    else:
        df = pd.read_csv(filename)
    return df

def wrangle_data():
    df = get_data()

    return df

In [ ]:
# if the repo been deleted after we get the repo name, wrangle_data code will stuck

In [17]:
df = wrangle_data()
df

,repo,language,readme_contents
0,cashapp/InflationInject,Kotlin,# Inflation Injection\n\nConstructor-inject vi...
1,InflationX/ViewPump,Kotlin,ViewPump\n========\n\nView inflation you can i...
2,rdeits/iris-distro,Matlab,Introduction\n============\n\nThis package con...
3,uhussain/WebCrawlerForOnlineInflation,Python,# Table of Contents \n1. [Introduction](README...
4,sandes/zipfly,Python,[![Build Status](https://img.shields.io/circle...
...,...,...,...
843,millerngit/RateOfInflation,C++,NaN
844,iamdingkai/what-s-driving-inflation,Jupyter Notebook,NaN
845,jrbsn/Inflation-Deep-Learning,NaN,# Inflation-Deep-Learning\nPredicts opening fo...
846,fscheler/Inflation_Tracker_World,HTML,NaN


In [18]:
df.shape

(848, 3)

In [19]:
df.head()

,repo,language,readme_contents
0,cashapp/InflationInject,Kotlin,# Inflation Injection\n\nConstructor-inject vi...
1,InflationX/ViewPump,Kotlin,ViewPump\n========\n\nView inflation you can i...
2,rdeits/iris-distro,Matlab,Introduction\n============\n\nThis package con...
3,uhussain/WebCrawlerForOnlineInflation,Python,# Table of Contents \n1. [Introduction](README...
4,sandes/zipfly,Python,[![Build Status](https://img.shields.io/circle...


In [22]:
df.to_csv("inflation_readme.csv")